In [12]:
# Source: https://www.quantbeckman.com/p/is-the-extra-return-worth-the-extra?utm_source=multiple-personal-recommendations-email&utm_medium=email&triedRedirect=true

def my_mean(data):
    """
    Calculates the arithmetic mean of a list of numbers.
    
    Parameters
    ----------
    data : list[float]
        List of numerical values
        
    Returns
    -------
    float
        The arithmetic mean of the input list
        Returns 0 if the input list is empty
        
    Notes
    -----
    - Handles empty lists by returning 0
    - Uses simple arithmetic mean calculation: sum(data)/len(data)
    """
    if not data:
        return 0
    return sum(data) / len(data)

def my_var(data):
    """
    Calculates the population variance of a list of numbers.
    
    Parameters
    ----------
    data : list[float]
        List of numerical values
        
    Returns
    -------
    float
        The population variance of the input list
        Returns 0 if the input list is empty
        
    Notes
    -----
    - Uses population variance formula: Σ(x - μ)²/N
    - Returns 0 for empty lists
    - Uses arithmetic mean from my_mean function
    """
    if not data:
        return 0
    m = my_mean(data)
    return sum((x - m) ** 2 for x in data) / len(data)

def my_std(data):
    """
    Calculates the population standard deviation of a list of numbers.
    
    Parameters
    ----------
    data : list[float]
        List of numerical values
        
    Returns
    -------
    float
        The population standard deviation of the input list
        Returns 0 if the input list is empty
        
    Notes
    -----
    - Calculated as square root of population variance
    - Returns 0 for empty lists
    - Uses my_var function for variance calculation
    """
    if not data:
        return 0
    return math.sqrt(my_var(data))

def my_percentile(data, percentile):
    """
    Computes the percentile of a list of numbers using linear interpolation.
    
    Parameters
    ----------
    data : list[float]
        List of numerical values
    percentile : float
        Percentile to compute (0-100)
        
    Returns
    -------
    float or None
        The interpolated value at the specified percentile
        Returns None if the input list is empty
        
    Notes
    -----
    - Uses linear interpolation between closest ranks
    - Handles edge cases (empty lists, exact matches)
    - Sorts data before computation
    """
    if not data:
        return None
    sorted_data = sorted(data)
    n = len(sorted_data)
    pos = (percentile / 100) * (n - 1)
    lower = math.floor(pos)
    upper = math.ceil(pos)
    if lower == upper:
        return sorted_data[int(pos)]
    lower_value = sorted_data[lower]
    upper_value = sorted_data[upper]
    weight = pos - lower
    return lower_value + weight * (upper_value - lower_value)

def cumulative_return(equity_initial, equity_final):
    """
    Calculates the total percentage return over the entire period.
    
    Parameters
    ----------
    equity_initial : float
        Initial equity value
    equity_final : float
        Final equity value
        
    Returns
    -------
    float
        The cumulative return as a percentage
        
    Notes
    -----
    - Formula: ((final/initial) - 1) * 100
    - Returns percentage value (e.g., 10.0 for 10% return)
    """
    return ((equity_final / equity_initial) - 1) * 100

def calculate_cagr(equity_initial, equity_final, years):
    """
    Calculates the Compound Annual Growth Rate (CAGR).
    
    Parameters
    ----------
    equity_initial : float
        Initial equity value
    equity_final : float
        Final equity value
    years : float
        Number of years between initial and final values
        
    Returns
    -------
    float or None
        The CAGR as a decimal
        Returns None if initial equity or years are <= 0
        
    Notes
    -----
    - Formula: (final/initial)^(1/years) - 1
    - Returns decimal value (e.g., 0.10 for 10% CAGR)
    - Handles invalid inputs (negative or zero values)
    """
    if equity_initial <= 0 or years <= 0:
        return None
    return (equity_final / equity_initial) ** (1 / years) - 1

def sharpe_ratio(returns, risk_free_rate):
    """
    Calculates the Sharpe ratio, which measures excess return per unit of risk.
    
    Parameters
    ----------
    returns : list[float]
        List of period returns
    risk_free_rate : float
        Risk-free rate for the period
        
    Returns
    -------
    float
        The Sharpe ratio
        Returns inf if standard deviation is zero
        
    Notes
    -----
    - Formula: (Mean(Returns - Rf)) / StdDev(Returns - Rf)
    - Uses excess returns over risk-free rate
    - Higher values indicate better risk-adjusted performance
    - Assumes returns are in decimal form
    """
    excess_returns = [r - risk_free_rate for r in returns]
    mean_excess = my_mean(excess_returns)
    std_excess = my_std(excess_returns)
    if std_excess == 0:
        return float('inf')
    return mean_excess / std_excess

def sortino_ratio(returns, risk_free_rate):
    """
    Calculates the Sortino ratio, focusing on downside deviation risk.
    
    Parameters
    ----------
    returns : list[float]
        List of period returns
    risk_free_rate : float
        Risk-free rate for the period
        
    Returns
    -------
    float
        The Sortino ratio
        
    Notes
    -----
    - Similar to Sharpe ratio but only penalizes downside volatility
    - Uses standard deviation of negative excess returns only
    - Higher values indicate better risk-adjusted performance
    - Uses small epsilon (1e-10) to avoid division by zero
    """
    excess_returns = [r - risk_free_rate for r in returns]
    downside_returns = [x for x in excess_returns if x < 0]
    downside_std = my_std(downside_returns) if downside_returns else 1e-10
    return my_mean(excess_returns) / downside_std

def omega_ratio(returns):
    """
    Calculates the Omega ratio, measuring probability-weighted ratio of gains vs losses.
    
    Parameters
    ----------
    returns : list[float]
        List of period returns
        
    Returns
    -------
    float
        The Omega ratio
        Returns inf if there are no negative returns
        
    Notes
    -----
    - Formula: sum(positive returns) / sum(|negative returns|)
    - Provides more information than mean-variance measures
    - Higher values indicate better performance
    - Returns infinity if there are no losses
    """
    sum_positive = sum(r for r in returns if r > 0)
    sum_negative = sum(abs(r) for r in returns if r < 0)
    if sum_negative == 0:
        return float('inf')
    return sum_positive / sum_negative

def annualized_volatility(returns, trading_days=252):
    """
    Calculates the annualized volatility of returns.
    
    Parameters
    ----------
    returns : list[float]
        List of period returns
    trading_days : int, optional
        Number of trading days in a year (default is 252)
        
    Returns
    -------
    float
        The annualized volatility
        
    Notes
    -----
    - Formula: daily_std * sqrt(trading_days)
    - Assumes returns are daily
    - Uses population standard deviation
    - Common values for trading_days: 252 (daily), 52 (weekly), 12 (monthly)
    """
    daily_vol = my_std(returns)
    return daily_vol * math.sqrt(trading_days)

def skewness(returns):
    """
    Calculates the skewness of returns distribution.
    
    Parameters
    ----------
    returns : list[float]
        List of period returns
        
    Returns
    -------
    float
        The skewness value
        Returns 0 for empty lists or zero standard deviation
        
    Notes
    -----
    - Measures asymmetry of returns distribution
    - Positive skew indicates right tail (extreme gains)
    - Negative skew indicates left tail (extreme losses)
    - Uses population formula
    """
    m = my_mean(returns)
    s = my_std(returns)
    n = len(returns)
    if n == 0 or s == 0:
        return 0
    skew_val = sum((r - m) ** 3 for r in returns) / n
    return skew_val / (s ** 3)

def kurtosis(returns):
    """
    Calculates the excess kurtosis of returns distribution.
    
    Parameters
    ----------
    returns : list[float]
        List of period returns
        
    Returns
    -------
    float
        The excess kurtosis value
        Returns 0 for empty lists or zero standard deviation
        
    Notes
    -----
    - Measures "tailedness" of returns distribution
    - Excess kurtosis = (kurtosis - 3) for comparison to normal distribution
    - Higher values indicate more extreme outliers
    - Uses population formula
    """
    m = my_mean(returns)
    s = my_std(returns)
    n = len(returns)
    if n == 0 or s == 0:
        return 0
    kurt = sum((r - m) ** 4 for r in returns) / n
    return kurt / (s ** 4) - 3

def var_metric(returns, confidence_level=0.95):
    """
    Calculates Value at Risk (VaR) at the given confidence level.
    
    Parameters
    ----------
    returns : list[float]
        List of period returns
    confidence_level : float, optional
        Confidence level for VaR calculation (default is 0.95)
        
    Returns
    -------
    float
        The VaR value at specified confidence level
        
    Notes
    -----
    - Historical VaR calculation method
    - Represents potential loss at given confidence level
    - Uses percentile function for calculation
    - Common confidence levels: 0.95, 0.99
    """
    percentile_value = (1 - confidence_level) * 100
    return my_percentile(returns, percentile_value)

def cvar_metric(returns, confidence_level=0.95):
    """
    Calculates Conditional Value at Risk (CVaR) or Expected Shortfall.
    
    Parameters
    ----------
    returns : list[float]
        List of period returns
    confidence_level : float, optional
        Confidence level for CVaR calculation (default is 0.95)
        
    Returns
    -------
    float
        The CVaR value at specified confidence level
        
    Notes
    -----
    - Also known as Expected Shortfall
    - Average loss beyond VaR
    - More sensitive to tail risk than VaR
    - Returns VaR if no returns beyond VaR threshold
    """
    var_val = var_metric(returns, confidence_level)
    tail_losses = [r for r in returns if r <= var_val]
    if tail_losses:
        return my_mean(tail_losses)
    else:
        return var_val

def covariance(x, y):
    """
    Calculates the population covariance between two lists.
    
    Parameters
    ----------
    x : list[float]
        First list of values
    y : list[float]
        Second list of values
        
    Returns
    -------
    float
        The covariance between x and y
        Returns 0 if lists are empty or of different lengths
        
    Notes
    -----
    - Formula: E[(X - μx)(Y - μy)]
    - Measures linear relationship between variables
    - Uses population formula
    - Requires equal length inputs
    """
    if len(x) != len(y) or not x:
        return 0
    m_x = my_mean(x)
    m_y = my_mean(y)
    return sum((xi - m_x) * (yi - m_y) for xi, yi in zip(x, y)) / len(x)

def calculate_beta(strategy_returns, benchmark_returns):
    """
    Calculates beta, measuring systematic risk relative to the benchmark.
    
    Parameters
    ----------
    strategy_returns : list[float]
        List of strategy returns
    benchmark_returns : list[float]
        List of benchmark returns
        
    Returns
    -------
    float
        The beta value
        Returns nan if benchmark variance is zero
        
    Notes
    -----
    - Formula: Covariance(strategy, benchmark) / Variance(benchmark)
    - Beta > 1 indicates higher volatility than benchmark
    - Beta < 1 indicates lower volatility than benchmark
    - Beta = 1 indicates same volatility as benchmark
    """
    var_benchmark = my_var(benchmark_returns)
    if var_benchmark == 0:
        return float('nan')
    return covariance(strategy_returns, benchmark_returns) / var_benchmark

def r_squared(strategy_returns, benchmark_returns):
    """
    Calculates R-squared, measuring goodness of fit to benchmark.
    
    Parameters
    ----------
    strategy_returns : list[float]
        List of strategy returns
    benchmark_returns : list[float]
        List of benchmark returns
        
    Returns
    -------
    float
        The R-squared value between 0 and 1
        Returns 0 if either series has zero standard deviation
        
    Notes
    -----
    - Square of correlation coefficient
    - Measures percentage of variance explained by benchmark
    - Values closer to 1 indicate better fit
    - Based on linear relationship assumption
    """
    cov = covariance(strategy_returns, benchmark_returns)
    std_strategy = my_std(strategy_returns)
    std_benchmark = my_std(benchmark_returns)
    if std_strategy == 0 or std_benchmark == 0:
        return 0
    corr = cov / (std_strategy * std_benchmark)
    return corr ** 2

def adjusted_r_squared(strategy_returns, benchmark_returns):
    """
    Calculates adjusted R-squared, penalizing for additional predictors.
    
    Parameters
    ----------
    strategy_returns : list[float]
        List of strategy returns
    benchmark_returns : list[float]
        List of benchmark returns
        
    Returns
    -------
    float
        The adjusted R-squared value
        Returns 0 if insufficient degrees of freedom
        
    Notes
    -----
    - Formula: 1 - [(1 - R²)(n-1)/(n-k-1)]
    - Penalizes for additional predictors
    - More conservative than regular R-squared
    - n is sample size, k is number of predictors (1 in this case)
    """
    r2 = r_squared(strategy_returns, benchmark_returns)
    n = len(strategy_returns)
    k = 1  # number of predictors (benchmark)
    if n <= k + 1:
        return 0
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))

def information_ratio(strategy_returns, benchmark_returns):
    """
    Calculates the Information Ratio, measuring active return per unit of active risk.
    
    Parameters
    ----------
    strategy_returns : list[float]
        List of strategy returns
    benchmark_returns : list[float]
        List of benchmark returns
        
    Returns
    -------
    float
        The Information Ratio
        Returns inf if tracking error is zero
        
    Notes
    -----
    - Formula: Mean(Active Returns) / StdDev(Active Returns)
    - Active returns = Strategy returns - Benchmark returns
    - Higher values indicate better risk-adjusted active returns
    - Similar to Sharpe ratio but uses benchmark instead of risk-free rate
    """
    active_returns = [s - b for s, b in zip(strategy_returns, benchmark_returns)]
    mean_active = my_mean(active_returns)
    std_active = my_std(active_returns)
    if std_active == 0:
        return float('inf')
    return mean_active / std_active

def treynor_ratio(strategy_return, risk_free_rate, beta):
    """
    Calculates the Treynor Ratio, measuring excess return per unit of systematic risk.
    
    Parameters
    ----------
    strategy_return : float
        Total return of the strategy
    risk_free_rate : float
        Risk-free rate
    beta : float
        Beta of the strategy relative to benchmark
        
    Returns
    -------
    float
        The Treynor Ratio
        Returns nan if beta is zero or nan
        
    Notes
    -----
    - Formula: (Strategy Return - Risk Free Rate) / Beta
    - Similar to Sharpe ratio but uses beta instead of standard deviation
    - Higher values indicate better risk-adjusted performance
    - Assumes beta accurately represents systematic risk
    """
    if beta == 0 or math.isnan(beta):
        return float('nan')
    return (strategy_return - risk_free_rate) / beta

def max_drawdown(equity_curve):
    """
    Calculates the maximum peak-to-trough decline in the equity curve.
    
    Parameters
    ----------
    equity_curve : list[float]
        List of equity values over time
        
    Returns
    -------
    float
        The maximum drawdown as a negative decimal
        
    Notes
    -----
    - Represents worst historical loss from peak to trough
    - Important measure of downside risk
    - Always negative or zero
    - Calculated using running maximum approach
    """
    peak = equity_curve[0]
    max_dd = 0
    for equity in equity_curve:
        if equity > peak:
            peak = equity
        drawdown = (equity - peak) / peak
        if drawdown < max_dd:
            max_dd = drawdown
    return max_dd

def longest_drawdown_days(equity_curve):
    """
    Calculates the longest consecutive period of drawdown in days.
    
    Parameters
    ----------
    equity_curve : list[float]
        List of equity values over time
        
    Returns
    -------
    int
        Number of consecutive days in longest drawdown period
        
    Notes
    -----
    - Measures persistence of drawdowns
    - Counts consecutive days below previous peak
    - Important for understanding recovery periods
    - Complements maximum drawdown metric
    """
    peak = equity_curve[0]
    longest = 0
    current = 0
    for equity in equity_curve:
        if equity < peak:
            current += 1
        else:
            current = 0
            peak = equity
        longest = max(longest, current)
    return longest

def average_drawdown_percentage(equity_curve):
    """
    Calculates the average drawdown percentage during drawdown periods.
    
    Parameters
    ----------
    equity_curve : list[float]
        List of equity values over time
        
    Returns
    -------
    float
        Average drawdown as a percentage
        Returns 0 if no drawdowns
        
    Notes
    -----
    - Provides typical drawdown magnitude
    - Includes all drawdown periods
    - Returns percentage value
    - Less sensitive to outliers than max drawdown
    """
    peak = equity_curve[0]
    drawdowns = []
    for equity in equity_curve:
        if equity > peak:
            peak = equity
        dd = (equity - peak) / peak
        if dd < 0:
            drawdowns.append(dd)
    return my_mean(drawdowns) * 100 if drawdowns else 0

def profit_factor(trade_results):
    """
    Calculates the Profit Factor: ratio of total profits to total losses.
    
    Parameters
    ----------
    trade_results : list[float]
        List of individual trade profits/losses
        
    Returns
    -------
    float
        The profit factor
        Returns inf if there are no losing trades
        
    Notes
    -----
    - Formula: sum(profits) / sum(|losses|)
    - Values > 1 indicate overall profitability
    - Higher values indicate better performance
    - Infinity indicates no losing trades
    """
    sum_wins = sum(trade for trade in trade_results if trade > 0)
    sum_losses = sum(abs(trade) for trade in trade_results if trade < 0)
    if sum_losses == 0:
        return float('inf')
    return sum_wins / sum_losses

def awal_ratio(trade_results):
    """
    Calculates the Average Win to Average Loss (AWAL) Ratio.
    
    Parameters
    ----------
    trade_results : list[float]
        List of individual trade profits/losses
        
    Returns
    -------
    float
        The AWAL ratio
        Returns inf if there are no losing trades
        
    Notes
    -----
    - Formula: average_win / |average_loss|
    - Different from profit factor (uses averages instead of sums)
    - Higher values indicate better risk/reward
    - Infinity indicates no losing trades
    """
    wins = [trade for trade in trade_results if trade > 0]
    losses = [trade for trade in trade_results if trade < 0]
    if not losses:
        return float('inf')
    avg_win = my_mean(wins) if wins else 0
    avg_loss = my_mean(losses) if losses else 0
    if avg_loss == 0:
        return float('inf')
    return avg_win / abs(avg_loss)

def calculate_expectancy(trade_results):
    """
    Calculates the expected value per trade.
    
    Parameters
    ----------
    trade_results : list[float]
        List of individual trade profits/losses
        
    Returns
    -------
    float
        The expected value per trade
        
    Notes
    -----
    - Formula: (win_prob * avg_win) + ((1 - win_prob) * avg_loss)
    - Combines win rate with average win/loss
    - Key metric for system profitability
    - Returns 0 for empty trade list
    """
    wins = [trade for trade in trade_results if trade > 0]
    losses = [trade for trade in trade_results if trade < 0]
    win_prob = len(wins) / len(trade_results) if trade_results else 0
    avg_win = my_mean(wins) if wins else 0
    avg_loss = my_mean(losses) if losses else 0
    return win_prob * avg_win + (1 - win_prob) * avg_loss

def rina_index(trade_results, max_drawdown):
    """
    Calculates the RINA Index, relating expectancy to maximum drawdown.
    
    Parameters
    ----------
    trade_results : list[float]
        List of individual trade profits/losses
    max_drawdown : float
        Maximum drawdown value (as a negative decimal)
        
    Returns
    -------
    float
        The RINA Index
        Returns inf if max_drawdown is zero
        
    Notes
    -----
    - Formula: Expectancy / |MaxDrawdown|
    - Relates system profitability to risk
    - Higher values indicate better risk-adjusted performance
    - Infinity indicates no drawdown
    """
    exp = calculate_expectancy(trade_results)
    if max_drawdown == 0:
        return float('inf')
    return exp / abs(max_drawdown)

def average_trade(trade_results):
    """
    Calculates the arithmetic mean of all trade results.
    
    Parameters
    ----------
    trade_results : list[float]
        List of individual trade profits/losses
        
    Returns
    -------
    float
        The average trade result
        Returns 0 for empty trade list
        
    Notes
    -----
    - Simple arithmetic mean of all trades
    - Includes both winning and losing trades
    - Positive value indicates overall profitability
    - Uses my_mean function
    """
    return my_mean(trade_results)

def winning_percentage(trade_results):
    """
    Calculates the percentage of winning trades.
    
    Parameters
    ----------
    trade_results : list[float]
        List of individual trade profits/losses
        
    Returns
    -------
    float
        The winning percentage (0-100)
        Returns 0 if no trades
        
    Notes
    -----
    - Formula: (number of winning trades / total trades) * 100
    - Winning trade defined as profit > 0
    - Returns percentage value (0-100)
    - Important but should not be viewed in isolation
    """
    wins = len([trade for trade in trade_results if trade > 0])
    total = len(trade_results)
    return (wins / total * 100) if total > 0 else 0

def time_under_water_percentage(equity_curve):
    """
    Calculates the percentage of time the equity curve is below its peak.
    
    Parameters
    ----------
    equity_curve : list[float]
        List of equity values over time
        
    Returns
    -------
    float
        Percentage of time spent in drawdown (0-100)
        
    Notes
    -----
    - Measures frequency of drawdowns
    - Important for psychological impact assessment
    - Returns percentage value (0-100)
    - Complements drawdown magnitude metrics
    """
    peak = equity_curve[0]
    under_water = 0
    for equity in equity_curve:
        if equity < peak:
            under_water += 1
        else:
            peak = equity
    return (under_water / len(equity_curve)) * 100

def max_consecutive_loss(trade_results):
    """
    Calculates the maximum number of consecutive losing trades.
    
    Parameters
    ----------
    trade_results : list[float]
        List of individual trade profits/losses
        
    Returns
    -------
    int
        Maximum number of consecutive losing trades
        
    Notes
    -----
    - Important for risk management
    - Helps in sizing position and risk limits
    - Indicates worst historical losing streak
    - Useful for psychological preparation
    """
    max_losses = 0
    current_losses = 0
    for trade in trade_results:
        if trade < 0:
            current_losses += 1
            max_losses = max(max_losses, current_losses)
        else:
            current_losses = 0
    return max_losses

def std_dev_trades(trade_results):
    """
    Calculates the standard deviation of trade outcomes.
    
    Parameters
    ----------
    trade_results : list[float]
        List of individual trade profits/losses
        
    Returns
    -------
    float
        Standard deviation of trade results
        Returns 0 for empty trade list
        
    Notes
    -----
    - Measures consistency of trading results
    - Uses population standard deviation
    - Lower values indicate more consistent returns
    - Uses my_std function
    """
    return my_std(trade_results)
